In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Import DataSet 'Data Analyst Jobs'

In [ ]:
# IMPORT DATA AND PACKED
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

In [ ]:
sns.set()
pd.options.plotting.backend = 'plotly'
plt.style.use('seaborn')
init_notebook_mode(connected = True)

In [ ]:
df = pd.read_csv('../input/data-analyst-jobs/DataAnalyst.csv')

In [ ]:
df.head()

# Data Cleaning to be Wokerd On

In [ ]:
# Delete column "Unnamed: 0"
df.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [ ]:
# Copy data a new DataFrame
df1 = df.copy()
df1.head()

In [ ]:
############################################### DATA CLEANING #############################################################

df1['Job Title'],df1['Department']= df['Job Title'].str.split(',',1).str
df1['Company Name'],_ = df['Company Name'].str.split('\n',1).str
df1['Salary Estimate'],_= df['Salary Estimate'].str.split('(',1).str
df1['Min Salary'],df1['Max Salary']= df1['Salary Estimate'].str.split('-').str

# Created values Max Salary and Min Salary as well as data cleaning with  strip methods from string class
df1['Min Salary'] = df1['Min Salary'].replace('', np.nan) # Replace withe spaces for nan values through the numpy
df1['Max Salary'] = df1['Max Salary'].replace('', np.nan)
df1['Min Salary'] = df1['Min Salary'].str.strip().str.lstrip('$').str.rstrip('K').fillna(0).astype(int)
df1['Max Salary'] = df1['Max Salary'].str.strip().str.lstrip('$').str.rstrip('K').fillna(0).astype(int)

In [ ]:
# Show 'df1' with the changes realized before 
df1.head()

In [ ]:
# Empty Field 'Salary Estimate' but created Max and Min salary before.
df1 = df1.drop(['Salary Estimate'],axis = 1)
df1.head()

In [ ]:
################################################ DATA ANALYTICS ##########################################################
# Count values from easy apply for each job offers
df1['Easy Apply'].value_counts()

# Companies Easy Apply for Data Analyst Jobs

In [ ]:
# Easy applicated only 80 companies, the rest could to be applicated dificult it.
# With function, it recode key and values to take a new values between 0 and 1
def recode(column, new_code):
    col_cod = pd.Series(column, copy = True)
    for key, values in new_code.items():
        col_cod.replace(key, values, inplace = True)
    return col_cod

In [ ]:
# Use new fuction
df1['Easy Apply'] = recode(df1['Easy Apply'],{'-1':0,'True':1})
df1['Competitors'] = recode(df1['Competitors'],{'-1':np.nan})

In [ ]:
df1.head()

In [ ]:
# Extracted data the Easy Apply with values equal '1', in the other words,  this part  will to make only in offers jobs where is easy applicated
df_easy_apply = df1[df1['Easy Apply']==1]
df_easy_apply.reset_index()
df_easy_apply.head()

In [ ]:
# New Dataframe through method 'Groupby' from 'pandas' packet. It's grouped in fallen order and count the values "Easy Apply" for each offer jobs
df_easy_apply_1 = df_easy_apply.groupby('Company Name')['Easy Apply'].count().reset_index()
df_easy_apply_1.head()

In [ ]:
# Apply order descendent for dataframe before and perform cut in ten primary
df_easy = df_easy_apply_1.sort_values('Easy Apply', ascending = False).head(10).reset_index().drop('index', axis = 1)
df_easy.head()

In [ ]:
# Plot grafic from DataFrom 'Easy Apply' for each company
plt.figure(figsize=(12,5))

chart = sns.barplot(data = df_easy, x = 'Company Name', y = 'Easy Apply')
chart = chart.set_xticklabels(chart.get_xticklabels(),
                             rotation = 60,
                             horizontalalignment = 'right',
                             fontweight = 'light')

# **Size Company - Employed Quantity that Offer Jobs as a Data Analyst**

In [ ]:
# Now, visualization of size company in funtion quantily employees that offers job in data analyst.

# Create DateFrame:
df_employed = df1['Size'].value_counts().to_frame().reset_index()
df_employed.head(10)

In [ ]:
# Empty rows with useless data
df_employed = df_employed.drop([6,8], axis = 0)
df_employed.head(10)

In [ ]:
# Change of field names for easy descriptive in graphic and visualization
df_employed = df_employed.rename(columns = {'index': 'Size Company - Employees Quantity',
                                            'Size': 'Nº Company'})
df_employed.shape
df_employed.head(8)

In [ ]:
# Finally, plot graphic barplot in horizontal for Size Company in function of employees Quantity:
# Use Seaborn
plt.figure(figsize = (10,6))

plot = sns.barplot(x = 'Nº Company', y = 'Size Company - Employees Quantity', data = df_employed)
plot = plot.set_xticklabels(plot.get_xticklabels(),
                           rotation = 65,
                           horizontalalignment = 'right',
                           fontweight = 'light')

# Classification of Companies vs Ratings

In [ ]:
# Use grouby method for get a new dataframe thad measure number companies your rating.
df_rating = df1.groupby('Rating')['Company Name'].count().to_frame().reset_index()
# the order descending in the ratings function
df_rating = df_rating.sort_values('Rating', ascending = False).reset_index()
df_rating = df_rating.drop(['index'], axis = 1)
df_rating.shape
df_rating.head()

In [ ]:
#Cut dataframe size, the only values that generate interesting is rating up at latest 3.0
df_rating = df_rating.iloc[:21,:]
df_rating.head()

In [ ]:
# Rename fields for this columns.
df_rating = df_rating.rename(columns = {'Company Name': 'Nº Companies'})
df_rating.head()

In [ ]:
# Plot visualization in graphic bars. 
plt.figure(figsize = (12,6))

plot = sns.barplot(x = 'Rating', y = 'Nº Companies', data = df_rating)
plot = plot.set_xticklabels(plot.get_xticklabels(),
                           horizontalalignment = 'right',
                           fontweight = 'light')

# Ownership Companies Type that Offers Jobs Data Analyst

In [ ]:
# Visualization of Ownerships companies type that offers job Data Analyst
df_ownership = df1['Type of ownership'].value_counts().to_frame()
df_ownership.head(15)


In [ ]:
# Empty row with '-1
df_ownership = df_ownership.drop('-1').reset_index()

In [ ]:
# Rename columns for their graphic
df_ownership = df_ownership.rename(columns = {'index': 'Type of Ownership','Type of ownership': 'Nº Offers Jobs'})
df_ownership.head()

In [ ]:
#Cut data until the row 9th
df_ownership = df_ownership.iloc[:9,:]
df_ownership.head()

In [ ]:
# Finaly, barplot with seaborn
plt.figure(figsize = (12,6))
plot = sns.barplot(x = 'Nº Offers Jobs', y = 'Type of Ownership', data = df_ownership)
plot = plot.set_xticklabels(plot.get_xticklabels(),
                           rotation = 65,
                           horizontalalignment = 'right')

# Range Salary for Each Sector

In [ ]:
# Now, analyze Max salary in fuction the each sector
df_max_salary = df1.groupby(['Sector']).mean().reset_index()
df_max_salary.head()

In [ ]:
# Eliminate rows innesesary 
df_max_salary = df_max_salary.drop([0,10,17,16], axis = 0)
df_max_salary = df_max_salary.drop(['Founded','Easy Apply'], axis = 1)
df_max_salary.head()

In [ ]:
# Order values in funtion 'Max Salary' and drop 'index'
df_max_salary = df_max_salary.sort_values('Max Salary', ascending = False).reset_index()
df_max_salary = df_max_salary.drop('index', axis = 1)
df_max_salary.head()

In [ ]:
# Visualization in bar graph with plotly

# Create Trace 1
trace1 = go.Bar(x = df_max_salary['Sector'],
                y = df_max_salary['Max Salary'],
                name = 'Max Salary',
                marker = dict(color ='rgb(55, 83, 109)',
                             line = dict(color = 'rgb(0,0,0)', width = 1.5)))

# Create Trace 2
trace2 = go.Bar(x = df_max_salary['Sector'],
                y = df_max_salary['Min Salary'],
                name = 'Min Salary',
                marker = dict(color = 'indianred',
                             line = dict(color = 'rgb(0,0,0)', width = 1.5)))

data = [trace1, trace2]

layout = go.Layout(title = 'Salary Range For Each Sector', 
                   barmode = 'group')

chart = go.Figure(data = data, layout = layout)
iplot(chart)


# Distribution of Job Offers as a Data Analyst in the Territory of the United States

In [ ]:
# Create a DataFrame with features (Mean Salary, Location and Mean Salary) 
df_states = pd.DataFrame()
dftemp = df1.copy()
dftemp['Mean Salary'] = (df1['Max Salary'] + df1['Min Salary']) / 2
df_states = pd.concat([df1['Sector'],df1['Location'], dftemp['Mean Salary']], axis = 1)
df_states.head()

In [ ]:
# Seperate Acronym and City from Location Field
df_states['City'], df_states['Acronym State'] = df_states['Location'].str.split(',', 1).str
# Remove field "Location":
df_states = df_states.drop('Location', axis = 1)
# Remove values '-1' into field "Sector":
df_states = df_states.drop(df_states[df_states['Sector'] == '-1'].index, axis = 0)
df_states.head()

In [ ]:
# Apply Groupy with DataFrame both in the count and the mean values, after concated fields in the new DataFrame:
df_states_group = df_states.groupby(['Acronym State']).count().reset_index()
df_salary_group = df_states.groupby(['Acronym State']).mean().reset_index()
# New Dataframe 
df_map = pd.DataFrame()
df_map = pd.concat([df_states_group['Acronym State'], df_states_group['Sector'], df_salary_group['Mean Salary']], axis = 1)
df_map.head()

In [ ]:
# Round Mean Salary Value with Lambda function and remove row in 'Araphone, CO' Value of "Sector" field
df_map = df_map.drop([1], axis = 0)
df_map['Mean Salary'] = df_map['Mean Salary'].apply(lambda x: round(x, 2))
# Rename Field
df_map = df_map.rename(columns = {'Sector': 'Nº Offers Jobs'})
df_map.head()

In [ ]:
# Plot Map of United States with Plotly in module graph_objects:
locations = list(df_map['Acronym State'])

# Remove withe space in "Acronym State" field
for i in range(0, len(locations)):
    locations[i] = locations[i].replace(" ", "")

qjobs = list(df_map['Nº Offers Jobs'])

# Create text that displays the basic data when the cursor is over the chart status 
df_map_str = df_map.copy()
for col in df_map_str.columns:
    df_map_str[col] = df_map_str[col].astype(str)
    
# Apply '$' symbol in Mean Salary values
df_map_str['Mean Salary'] = df_map_str['Mean Salary'].apply(lambda x: "$" + x + "/Year")
# Text visualization:
df_map_str['Text'] = df_map_str['Acronym State']  + '<br>' + \
    "Nª Offers Jobs: " + df_map_str['Nº Offers Jobs'] + '<br>' \
        "Mean Salary from the Jobs as Data Analyst: " + '<br>' + df_map_str['Mean Salary']
df_map_str.head()

In [ ]:
# Plot Map in plotly
import plotly.graph_objects as go


fig = go.Figure(data = go.Choropleth(locations = locations,
                                    z = qjobs,
                                    locationmode = 'USA-states',
                                    text = df_map_str['Text'],
                                    marker_line_color = 'black',
                                    colorbar_title = 'Nº Offers Jobs'),) 

fig.update_layout(title_text = 'Nº Offers Jobs as Data Analyst<br>United States of America',
                 geo = dict(scope = 'usa',
                           projection = go.layout.geo.Projection(type = 'albers usa'),
                           showlakes = True,
                           lakecolor = 'rgb(255,255,255)'),)
fig.show()

# Cluster Analysis (K-Means) and PCA

In [ ]:
# A cluster study is performed in a data frame for the data in order to find similarities
# In addition, this is supported with PCA study
# Import Library
from sklearn.decomposition import PCA # For PCA dimensionality reduction
from sklearn.cluster import KMeans    # Method for Cluster Kmeans
from sklearn.preprocessing import QuantileTransformer #Realize Scaler Data
# For visualization from 3D plot
import plotly.express as px

In [ ]:
# Create Data Frame with "Mean Salary, Founded Years and Rating" for cluster method and PCA
df2 = df1.copy()
df2 = df2.drop(df2[df2['Founded'] == -1].index)
df2 = df2.drop(df2[df2['Rating'] == -1].index)
df2['Years Founded'] = (2020 - df2['Founded'])
df2['Mean Salary'] = (df2['Max Salary'] + df2['Min Salary']) / 2

# Create DataFrame
df3 = pd.DataFrame()
df3 = pd.concat([df2['Rating'], df2['Mean Salary'], df2['Years Founded']], axis = 1)
df3.head()

In [ ]:
# Rescale values with QuantileTransformer class
df3_scaler = QuantileTransformer().fit_transform(df3)
print(df3_scaler)

In [ ]:
# PCA calculation is performed with a reduction of dimensionality equal to 2
pca = PCA(n_components = 2)
df_component = pca.fit_transform(df3_scaler)
print(df_component.shape, df3_scaler.shape)

In [ ]:
# The number cluster is determined
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', n_init = 10, 
                   max_iter = 300, random_state = 0)
    kmeans.fit(df3_scaler)
    wcss.append(kmeans.inertia_)

In [ ]:
#Plot curve inertian cluster
fig = plt.figure(figsize = (8,7))
plt.plot(range(1,11), wcss)
plt.scatter(3, wcss[2], c = 'red',s = 200)
plt.text(3 + 0.4, wcss[2], s = '3 - Clusters', fontsize = 14)
plt.xlabel('Nº Clusters', fontsize = 14)
plt.ylabel('Inertian Infraclusters', fontsize = 14)
plt.title('Elbow Method', fontsize = 14)

In [ ]:
# 4 clusters are selected, at that elbow point an optimized cluster can be achieved.
kmeans = KMeans(n_clusters = 3, init = 'k-means++', n_init = 10, 
                max_iter = 300, random_state = 0)
kmeans.fit(df3_scaler)
y_predict = kmeans.predict(df3_scaler)
print(y_predict)

In [ ]:
# First one, 3D visualization are better for show clusters calculated

fig3d = px.scatter_3d(df3, x = 'Rating', y = 'Mean Salary', z = 'Years Founded', color = y_predict,
                     opacity = 0.80, size_max = 8)

fig3d.update_layout(margin=dict(l=0, r=0, b=0, t=0))

fig3d.show()

In [ ]:
# Graph PCA component with cluster predict
dfsns = pd.DataFrame(df_component)
# Rename Columns:
dfsns = dfsns.rename(columns = {0: 'Component 1', 1: 'Component 2'})

#Plot with Seaborn
sns.set_style('darkgrid')
sns.relplot(x = 'Component 1', y = 'Component 2', hue = y_predict, data = dfsns, palette= 'Dark2')

In [ ]:
df_predict = pd.DataFrame(y_predict)
df_predict = df_predict.rename(columns = {0 : 'Cluster'})
df_predict.head()

In [ ]:
# Dataframe with cluster classification
df3_scaler = pd.DataFrame(df3_scaler)
df3_scaler = df3_scaler.rename(columns = {0: 'Rating', 1: 'Mean Salary', 2: 'Years Founded'})
df3_scaler['Cluster'] = df_predict['Cluster']
df3_scaler['Cluster'] = recode(df3_scaler['Cluster'], 
                               {0: 'Cluster-1', 
                                1: 'Cluster-2', 
                                2: 'Cluster-3'})
df3_scaler.head()

In [ ]:
# Groupby apply
df_cluster_group = df3_scaler.groupby('Cluster').mean()
df_cluster_group.head()

In [ ]:
# Graph scatter bar for each cluster number
# Scaler values

colors = ['#CF8EF8', '#523C81', '#D2D132']
y = np.array(df_cluster_group)
plt.figure(1, figsize = (18,8))
#Cluster 1
plt.subplot(1,3,1)
value = y[:1,:].tolist()[0]
plt.bar(range(len(value)), height = value, width = 1/1.5, color = colors)
plt.xticks(range(df_cluster_group.shape[1]),df_cluster_group.columns)
#Cluster 2
plt.subplot(1,3,2)
value = y[1:2,:].tolist()[0]
plt.bar(range(len(value)), height = value, width = 1/1.5, color = colors)
plt.xticks(range(df_cluster_group.shape[1]),df_cluster_group.columns)
#Cluster 3
plt.subplot(1,3,3)
value = y[2:3,:].tolist()[0]
plt.bar(range(len(value)), height = value, width = 1/1.5, color = colors)
plt.xticks(range(df_cluster_group.shape[1]),df_cluster_group.columns)

In [ ]:
print(max(df2['Mean Salary']))